In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

## Загрузка данных

In [2]:
data = pd.read_csv('./data/amine_treatment-final.csv', on_bad_lines='skip')
data

,"feed_gas temperature, C","feed_gas pressure, kPa","feed_gas mass flow, kg/h",feed_gas molecular weight,"feed_gas Mass density, kg/m3","feed_gas actual liquid flow, m3/h","feed_gas actual vapour flow, m3/h",feed_gas CO2 mol frac,feed_gas Methane mol frac,feed_gas Ethane mol frac,...,degased_amine Propane mol frac,degased_amine i-Butane mol frac,degased_amine n-Butane mol frac,degased_amine i-Pentane mol frac,degased_amine n-Pentane mol frac,degased_amine H2S mol frac,degased_amine H2O mol frac,degased_amine MDEAmine mol frac,degased_amine H2S ppm,degased_amine CO2 ppm
0,52.0,6701.325,259647.0,24.668523,80.705482,0.000000,3217.216383,0.031875,0.633122,0.192729,...,0.000004,1.979803e-06,6.785930e-06,3.039502e-08,1.629357e-07,0.014838,0.910916,0.062270,22521.880092,23245.607009
1,40.0,6701.325,63853.0,23.411939,79.292250,0.000000,805.286774,0.039520,0.681947,0.179992,...,0.000006,1.099164e-07,7.886635e-07,2.425456e-06,2.679624e-08,0.010470,0.896612,0.077190,15891.219056,30468.484150
2,46.0,6701.325,15928.0,27.803853,108.358884,0.000000,146.993024,0.027859,0.577870,0.163095,...,0.000018,5.179434e-05,2.863001e-05,4.721545e-06,1.359401e-06,0.001284,0.860268,0.132676,1949.372531,10648.306282
3,36.0,6701.325,150276.0,22.053344,74.092459,0.000000,2028.222609,0.005420,0.856717,0.000249,...,0.000039,8.734358e-06,3.706189e-05,5.426945e-05,3.949909e-06,0.001233,0.809722,0.184359,1871.528366,8438.736973
4,12.0,6701.325,52691.0,24.334510,114.182302,0.002870,461.460940,0.027297,0.694634,0.091063,...,0.000100,7.719691e-05,2.661312e-05,1.351350e-05,6.619552e-06,0.003300,0.788340,0.200715,5008.207346,14011.150105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512795,18.0,6701.325,278813.0,27.267819,147.437736,0.239690,1890.816202,0.036621,0.666807,0.042159,...,0.000053,1.226987e-04,3.336251e-05,4.145371e-06,8.510407e-06,0.002101,0.796328,0.187149,3188.443718,27411.897443
512796,33.0,6701.325,72612.0,26.629816,112.530303,0.000000,645.266193,0.025383,0.605142,0.150361,...,0.000049,8.271254e-05,1.199104e-05,1.810930e-06,4.749955e-06,0.003784,0.826966,0.162727,5742.847844,11941.343733
512797,17.0,6701.325,272718.0,26.290335,135.390018,0.133398,2014.180541,0.039356,0.606850,0.167318,...,0.000037,2.874297e-05,3.343060e-06,1.045919e-05,2.187381e-06,0.045335,0.725301,0.179546,68812.636471,97071.672175
512798,59.0,6701.325,75970.0,24.231673,73.727565,0.000000,1030.415146,0.037951,0.732049,0.086286,...,0.000013,1.782460e-05,8.286743e-06,5.417825e-06,2.393693e-07,0.002551,0.877567,0.106280,3871.441495,26287.366187


In [3]:
print(data["lean_amine pressure, kPa"].min())
print(data["lean_amine pressure, kPa"].max())

6701.325
6701.325


# Работаем над определением физических свойств входящих потоков

In [4]:
stream_physic = data.sample(frac=0.85)
len(stream_physic)

435880

In [5]:
list(stream_physic.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

#### Получаем обучающую и тестовую выборку для определения физических свойств сырьевых потоков

In [6]:
feed_streams_physic = stream_physic[['feed_gas temperature, C', 'feed_gas pressure, kPa', 'feed_gas mass flow, kg/h',
                                 'feed_gas molecular weight', 'feed_gas Mass density, kg/m3', 'feed_gas actual liquid flow, m3/h',
                                 'feed_gas actual vapour flow, m3/h', 'feed_gas CO2 mol frac', 'feed_gas Methane mol frac',
                                 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
                                 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
                                 'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
                                 'lean_amine temperature, C', 'lean_amine pressure, kPa', 'lean_amine mass flow, kg/h',
                                 'lean_amine molecular weight', 'lean_amine Mass density, kg/m3',
                                 'lean_amine actual liquid flow, m3/h', 'lean_amine actual vapour flow, m3/h',
                                 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
                                 'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac',
                                 'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac',
                                 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac', 'lean_amine H2O mol frac',
                                 'lean_amine MDEAmine mol frac'
                                ]]

# Создаем X и y
X = feed_streams_physic[['feed_gas temperature, C', 'feed_gas pressure, kPa', 'feed_gas mass flow, kg/h',
                    'feed_gas CO2 mol frac', 'feed_gas Methane mol frac','feed_gas Ethane mol frac',
                    'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
                    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
                    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
                    'lean_amine temperature, C', 'lean_amine pressure, kPa', 'lean_amine mass flow, kg/h',
                    'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
                    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
                    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac',
                    'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac'
                    ]]

y = feed_streams_physic.drop(['feed_gas temperature, C', 'feed_gas pressure, kPa', 'feed_gas mass flow, kg/h',
                    'feed_gas CO2 mol frac', 'feed_gas Methane mol frac','feed_gas Ethane mol frac',
                    'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
                    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
                    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
                    'lean_amine temperature, C', 'lean_amine pressure, kPa', 'lean_amine mass flow, kg/h',
                    'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
                    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
                    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac',
                    'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac'
                    ], axis=1)

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), ['feed_gas temperature, C', 'feed_gas pressure, kPa', 'feed_gas mass flow, kg/h',
                    'feed_gas CO2 mol frac', 'feed_gas Methane mol frac','feed_gas Ethane mol frac',
                    'feed_gas Propane mol frac', 'feed_gas i-Butane mol frac',
                    'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac', 'feed_gas n-Pentane mol frac',
                    'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
                    'lean_amine temperature, C', 'lean_amine pressure, kPa', 'lean_amine mass flow, kg/h',
                    'lean_amine CO2 mol frac', 'lean_amine Methane mol frac', 'lean_amine Ethane mol frac',
                    'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac', 'lean_amine n-Butane mol frac',
                    'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac', 'lean_amine H2S mol frac',
                    'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac'
                     ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)
print(X_test_normal.shape)
print(y_test)

(87176, 28)
        feed_gas molecular weight  feed_gas Mass density, kg/m3  \
484815                  24.327612                    116.545865   
294539                  29.652080                    206.415922   
145805                  26.423588                    101.737807   
469422                  27.225562                    183.333369   
121198                  24.270627                    110.593090   
...                           ...                           ...   
495695                  31.859709                    154.905526   
151541                  26.984838                    141.443529   
57790                   25.252106                    106.865799   
278081                  26.333895                    120.563038   
461745                  26.452068                     89.868243   

        feed_gas actual liquid flow, m3/h  feed_gas actual vapour flow, m3/h  \
484815                           0.070854                        1882.895989   
294539                 

#### Нейронная сеть для определения физических свойств сырьевых потоков

In [50]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
feed_streams_physic_model = tf.keras.Sequential([
    tf.keras.layers.Dense(28, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(8)
])

# 2. Compile the model
feed_streams_physic_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
feed_streams_physic_model.fit(X_train_normal, y_train, epochs=50)

Epoch 1/50
10897/10897 [==============================] - 33s 3ms/step - loss: 30.2872 - mae: 30.2872
Epoch 2/50
10897/10897 [==============================] - 31s 3ms/step - loss: 4.1367 - mae: 4.1367
Epoch 3/50
10897/10897 [==============================] - 34s 3ms/step - loss: 3.5491 - mae: 3.5491
Epoch 4/50
10897/10897 [==============================] - 31s 3ms/step - loss: 3.3291 - mae: 3.3291
Epoch 5/50
10897/10897 [==============================] - 32s 3ms/step - loss: 3.1010 - mae: 3.1010
Epoch 6/50
10897/10897 [==============================] - 32s 3ms/step - loss: 2.8532 - mae: 2.8532
Epoch 7/50
10897/10897 [==============================] - 32s 3ms/step - loss: 2.5848 - mae: 2.5848
Epoch 8/50
10897/10897 [==============================] - 32s 3ms/step - loss: 2.4597 - mae: 2.4597
Epoch 9/50
10897/10897 [==============================] - 32s 3ms/step - loss: 2.3818 - mae: 2.3818
Epoch 10/50
10897/10897 [==============================] - 28s 3ms/step - loss: 2.3233 - mae: 2.32

In [51]:
feed_streams_physic_model.evaluate(X_test_normal)

2725/2725 [==============================] - 3s 1ms/step - loss: 0.0000e+00 - mae: 0.0000e+00


[0.0, 0.0]

In [52]:
y_pred = feed_streams_physic_model.predict(X_test_normal)

2725/2725 [==============================] - 7s 2ms/step


In [53]:
print(y_pred[1])

[2.6205870e+01 1.1732221e+02 3.1152061e-01 1.3183557e+03 3.0197891e+01
 1.0324226e+03 4.4165005e+02 2.7216086e-03]


In [54]:
y_test

,feed_gas molecular weight,"feed_gas Mass density, kg/m3","feed_gas actual liquid flow, m3/h","feed_gas actual vapour flow, m3/h",lean_amine molecular weight,"lean_amine Mass density, kg/m3","lean_amine actual liquid flow, m3/h","lean_amine actual vapour flow, m3/h"
171617,25.575780,90.016785,0.000000,329.383013,34.410598,1040.681338,57.137567,0
16861,26.488511,116.712950,0.004574,1324.612784,30.125225,1033.310888,439.304381,0
493853,23.692426,94.591521,0.000000,1162.894930,32.586751,1034.903152,420.628732,0
118877,26.037843,115.285355,0.225034,2326.098205,31.284679,1046.699413,404.054875,0
496055,25.880592,138.667693,0.021220,231.373703,35.046473,1053.974734,261.437956,0
...,...,...,...,...,...,...,...,...
422010,23.992865,100.351805,0.000000,2687.066762,38.189432,1059.005370,640.439623,0
231510,26.089645,113.092805,0.049926,1184.490507,31.163168,1044.251055,237.820205,0
266373,25.895448,109.882047,0.102067,2296.715362,21.529358,1029.360525,659.855302,0
419207,25.687327,132.709191,0.275366,1867.070810,29.917277,1046.566643,325.533976,0


#### Сохраняем модель для предсказания физических свойств сырьевых потоков

In [56]:
feed_streams_physic_model.save('./models/amine_treatment/amine_treatment_feed_streams_physic_model.h5')

In [57]:
feed_streams_physic_model.save('./models/amine_treatment/amine_treatment_feed_streams_physic_model_sv')

INFO:tensorflow:Assets written to: ./models/amine_treatment/amine_treatment_feed_streams_physic_model_sv\assets


INFO:tensorflow:Assets written to: ./models/amine_treatment/amine_treatment_feed_streams_physic_model_sv\assets


# Работаем над определением составов продуктовых потоков

### Получаем отдельную выборку данных для определения составов продуктов

In [7]:
rounded_composition = data
columns = [ 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane mol frac',
 'lean_amine n-Butane mol frac',
 'lean_amine i-Pentane mol frac',
 'lean_amine n-Pentane mol frac',
 'lean_amine H2S mol frac',
 'lean_amine H2O mol frac',
 'lean_amine MDEAmine mol frac',
 'sweet_gas CO2 mol frac',
 'sweet_gas Methane mol frac',
 'sweet_gas Ethane mol frac',
 'sweet_gas Propane mol frac',
 'sweet_gas i-Butane mol frac',
 'sweet_gas n-Butane mol frac',
 'sweet_gas i-Pentane mol frac',
 'sweet_gas n-Pentane mol frac',
 'sweet_gas H2S mol frac',
 'sweet_gas H2O mol frac',
 'sweet_gas MDEAmine mol frac',
 'rich_amine CO2 mol frac',
 'rich_amine Methane mol frac',
 'rich_amine Ethane mol frac',
 'rich_amine Propane mol frac',
 'rich_amine i-Butane mol frac',
 'rich_amine n-Butane mol frac',
 'rich_amine i-Pentane mol frac',
 'rich_amine n-Pentane mol frac',
 'rich_amine H2S mol frac',
 'rich_amine H2O mol frac',
 'rich_amine MDEAmine mol frac']

for col in columns:
    rounded_composition[col] = rounded_composition[col].apply(lambda x: 0 if x < 0.0001 else x)

In [8]:
rounded_composition_frac = rounded_composition.sample(frac=0.85)
len(rounded_composition_frac)

435880

#### Получаем обучающую и тестовую выборку

In [21]:
mol_composition = rounded_composition_frac[['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane mol frac',
 'lean_amine n-Butane mol frac',
 'lean_amine i-Pentane mol frac',
 'lean_amine n-Pentane mol frac',
 'lean_amine H2S mol frac',
 'lean_amine H2O mol frac',
 'lean_amine MDEAmine mol frac',
 'lean_amine H2S ppm',
 'lean_amine CO2 ppm',
'sweet_gas temperature, C',
'sweet_gas mass flow, kg/h',
'sweet_gas CO2 mol frac',
 'sweet_gas Methane mol frac',
 'sweet_gas Ethane mol frac',
 'sweet_gas Propane mol frac',
 'sweet_gas i-Butane mol frac',
 'sweet_gas n-Butane mol frac',
 'sweet_gas i-Pentane mol frac',
 'sweet_gas n-Pentane mol frac',
 'sweet_gas H2S mol frac',
 'sweet_gas H2O mol frac',
 'sweet_gas MDEAmine mol frac',
 'sweet_gas H2S ppm',
    'sweet_gas CO2 ppm',
 'rich_amine temperature, C',
 'rich_amine mass flow, kg/h', 
 'rich_amine CO2 mol frac',
 'rich_amine Methane mol frac',
 'rich_amine Ethane mol frac',
 'rich_amine Propane mol frac',
 'rich_amine i-Butane mol frac',
 'rich_amine n-Butane mol frac',
 'rich_amine i-Pentane mol frac',
 'rich_amine n-Pentane mol frac',
 'rich_amine H2S mol frac',
 'rich_amine H2O mol frac',
 'rich_amine MDEAmine mol frac',
 'rich_amine H2S ppm',
 'rich_amine CO2 ppm',]]

# Создаем X и y
stream_names_to_remove = ['sweet_gas', 'rich_amine']
pattern = '|'.join(stream_names_to_remove)
cols_to_remove = mol_composition.filter(regex=pattern).columns

X = mol_composition.drop(cols_to_remove, axis=1)
y = mol_composition[['sweet_gas CO2 mol frac',
 'sweet_gas Methane mol frac',
 'sweet_gas Ethane mol frac',
 'sweet_gas Propane mol frac',
 'sweet_gas i-Butane mol frac',
 'sweet_gas n-Butane mol frac',
 'sweet_gas i-Pentane mol frac',
 'sweet_gas n-Pentane mol frac',
 'sweet_gas H2S mol frac',
 'sweet_gas H2O mol frac',
 'sweet_gas MDEAmine mol frac',
 'rich_amine CO2 mol frac',
 'rich_amine Methane mol frac',
 'rich_amine Ethane mol frac',
 'rich_amine Propane mol frac',
 'rich_amine i-Butane mol frac',
 'rich_amine n-Butane mol frac',
 'rich_amine i-Pentane mol frac',
 'rich_amine n-Pentane mol frac',
 'rich_amine H2S mol frac',
 'rich_amine H2O mol frac',
 'rich_amine MDEAmine mol frac']]

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), ['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane mol frac',
 'lean_amine n-Butane mol frac',
 'lean_amine i-Pentane mol frac',
 'lean_amine n-Pentane mol frac',
 'lean_amine H2S mol frac',
 'lean_amine H2O mol frac',
 'lean_amine MDEAmine mol frac',
 'lean_amine H2S ppm',
 'lean_amine CO2 ppm',])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)
X_train_normal.shape, y_train.shape, X_train

((348704, 40),
 (348704, 22),
         feed_gas temperature, C  feed_gas pressure, kPa  \
 213732                     42.0                6701.325   
 84229                      27.0                6701.325   
 223924                     54.0                6701.325   
 227302                     47.0                6701.325   
 310436                     26.0                6701.325   
 ...                         ...                     ...   
 349531                     28.0                6701.325   
 414316                     34.0                6701.325   
 13364                      32.0                6701.325   
 481559                     20.0                6701.325   
 138308                     43.0                6701.325   
 
         feed_gas mass flow, kg/h  feed_gas molecular weight  \
 213732                   72083.0                  23.552553   
 84229                   222634.0                  24.772831   
 223924                  259993.0                  26.13

#### Нейронная сеть для определения состава продуктов

In [19]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_composition_model = tf.keras.Sequential([
    tf.keras.layers.Dense(40, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(22)
])

# 2. Compile the model
prod_composition_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    metrics=['mae']
)

# 3. Fit the model
prod_composition_model.fit(X_train_normal, y_train, epochs=10)

Epoch 1/10
10897/10897 [==============================] - 25s 2ms/step - loss: 0.0032 - mae: 0.0032
Epoch 2/10
10897/10897 [==============================] - 27s 3ms/step - loss: 0.0015 - mae: 0.0015
Epoch 3/10
10897/10897 [==============================] - 26s 2ms/step - loss: 0.0014 - mae: 0.0014
Epoch 4/10
10897/10897 [==============================] - 28s 3ms/step - loss: 0.0013 - mae: 0.0013
Epoch 5/10
10897/10897 [==============================] - 28s 3ms/step - loss: 0.0012 - mae: 0.0012
Epoch 6/10
10897/10897 [==============================] - 29s 3ms/step - loss: 0.0011 - mae: 0.0011
Epoch 7/10
10897/10897 [==============================] - 28s 3ms/step - loss: 0.0011 - mae: 0.0011
Epoch 8/10
10897/10897 [==============================] - 26s 2ms/step - loss: 0.0010 - mae: 0.0010
Epoch 9/10
10897/10897 [==============================] - 25s 2ms/step - loss: 0.0010 - mae: 0.0010
Epoch 10/10
10897/10897 [==============================] - 25s 2ms/step - loss: 9.9188e-04 - mae: 9.

In [23]:
prod_composition_model.evaluate(X_test_normal)

2725/2725 [==============================] - 4s 933us/step - loss: 0.0000e+00 - mae: 0.0000e+00


[0.0, 0.0]

In [24]:
y_pred = prod_composition_model.predict(X_test_normal)

2725/2725 [==============================] - 3s 1ms/step


In [27]:
print(y_pred[0])
print(' ')
print(sum(y_pred[0]))

[ 6.06644107e-03  7.72401869e-01  5.09823635e-02  3.90587151e-02
  1.58503409e-02  6.38498217e-02  4.04703096e-02  8.42980295e-03
 -5.54386061e-05  8.71160650e-04 -4.03115060e-04  8.33516754e-03
  1.40478741e-03 -2.26614909e-04 -2.64295377e-04 -3.10746487e-04
  4.20355238e-04 -4.65616933e-04  6.13255892e-04  1.62505060e-02
  8.60322595e-01  1.10914335e-01]
 
1.9945159988419618


In [29]:
y_test

,sweet_gas CO2 mol frac,sweet_gas Methane mol frac,sweet_gas Ethane mol frac,sweet_gas Propane mol frac,sweet_gas i-Butane mol frac,sweet_gas n-Butane mol frac,sweet_gas i-Pentane mol frac,sweet_gas n-Pentane mol frac,sweet_gas H2S mol frac,sweet_gas H2O mol frac,...,rich_amine Methane mol frac,rich_amine Ethane mol frac,rich_amine Propane mol frac,rich_amine i-Butane mol frac,rich_amine n-Butane mol frac,rich_amine i-Pentane mol frac,rich_amine n-Pentane mol frac,rich_amine H2S mol frac,rich_amine H2O mol frac,rich_amine MDEAmine mol frac
339245,0.008085,0.774453,0.049807,0.039194,0.015421,0.062568,0.040902,0.008717,0.0,0.000847,...,0.001389,0.000125,0.000000,0.000000,0.000000,0.000000,0.0,0.017366,0.861645,0.111186
250174,0.002344,0.641220,0.144709,0.029324,0.047782,0.101133,0.028394,0.004716,0.0,0.000375,...,0.002042,0.000402,0.000000,0.000000,0.000134,0.000000,0.0,0.000308,0.816485,0.161787
133045,0.003660,0.738876,0.083866,0.051539,0.056691,0.004079,0.057624,0.003312,0.0,0.000353,...,0.001695,0.000187,0.000000,0.000000,0.000000,0.000000,0.0,0.000725,0.906283,0.085725
333337,0.004922,0.637382,0.150184,0.092465,0.045317,0.047471,0.016767,0.003732,0.0,0.001745,...,0.001495,0.000606,0.000292,0.000209,0.000119,0.000000,0.0,0.004032,0.806807,0.179725
95608,0.005534,0.626211,0.122019,0.099335,0.052004,0.047979,0.042177,0.004179,0.0,0.000558,...,0.002034,0.000448,0.000264,0.000172,0.000101,0.000000,0.0,0.003050,0.770792,0.213953
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
446306,0.000540,0.703666,0.047900,0.062562,0.060220,0.079400,0.034328,0.009441,0.0,0.001923,...,0.001682,0.000205,0.000200,0.000292,0.000192,0.000000,0.0,0.000738,0.811250,0.181637
256375,0.003716,0.631033,0.166902,0.065208,0.020335,0.083919,0.022221,0.006340,0.0,0.000323,...,0.001379,0.000565,0.000167,0.000000,0.000168,0.000000,0.0,0.019551,0.790957,0.161553
275002,0.010912,0.678415,0.076600,0.088542,0.006894,0.096690,0.032623,0.008363,0.0,0.000920,...,0.001699,0.000482,0.000480,0.000000,0.000426,0.000205,0.0,0.031082,0.713492,0.241682
267497,0.003556,0.578801,0.138035,0.085653,0.101877,0.083826,0.000774,0.006586,0.0,0.000882,...,0.001248,0.000294,0.000109,0.000133,0.000000,0.000000,0.0,0.003532,0.888950,0.100331


In [30]:
prod_composition_model.layers[0].input_shape

(None, 40)

#### Сохраняем модель для предсказания состава продуктовых потоков

In [32]:
prod_composition_model.save('./models/amine_treatment/amine_treatment_prod_composition_model.h5')

In [34]:
prod_composition_model.save('./models/amine_treatment/amine_treatment_prod_composition_model_sv')

INFO:tensorflow:Assets written to: ./models/amine_treatment/amine_treatment_prod_composition_model_sv\assets


INFO:tensorflow:Assets written to: ./models/amine_treatment/amine_treatment_prod_composition_model_sv\assets


# Работаем над определением температур продуктовых потоков

In [4]:
stream_physic = data.sample(frac=0.9)
len(stream_physic)

461520

In [5]:
list(stream_physic.columns)

['feed_gas temperature, C',
 'feed_gas pressure, kPa',
 'feed_gas mass flow, kg/h',
 'feed_gas molecular weight',
 'feed_gas Mass density, kg/m3',
 'feed_gas actual liquid flow, m3/h',
 'feed_gas actual vapour flow, m3/h',
 'feed_gas CO2 mol frac',
 'feed_gas Methane mol frac',
 'feed_gas Ethane mol frac',
 'feed_gas Propane mol frac',
 'feed_gas i-Butane mol frac',
 'feed_gas n-Butane mol frac',
 'feed_gas i-Pentane mol frac',
 'feed_gas n-Pentane mol frac',
 'feed_gas H2S mol frac',
 'feed_gas H2O mol frac',
 'feed_gas MDEAmine mol frac',
 'feed_gas H2S ppm',
 'feed_gas CO2 ppm',
 'Absorber number of stages',
 'lean_amine temperature, C',
 'lean_amine pressure, kPa',
 'lean_amine mass flow, kg/h',
 'lean_amine molecular weight',
 'lean_amine Mass density, kg/m3',
 'lean_amine actual liquid flow, m3/h',
 'lean_amine actual vapour flow, m3/h',
 'lean_amine CO2 mol frac',
 'lean_amine Methane mol frac',
 'lean_amine Ethane mol frac',
 'lean_amine Propane mol frac',
 'lean_amine i-Butane

#### Создаем выборку для определения температур продуктовых потоков

In [6]:
prod_streams_physic = stream_physic[
    [
        'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight',
        'feed_gas Mass density, kg/m3', 'feed_gas actual liquid flow, m3/h', 'feed_gas actual vapour flow, m3/h',
        'feed_gas CO2 mol frac', 'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac',
        'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
        'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
        'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
        'lean_amine molecular weight', 'lean_amine Mass density, kg/m3', 'lean_amine actual liquid flow, m3/h',
        'lean_amine actual vapour flow, m3/h', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac',
        'lean_amine Ethane mol frac', 'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac',
        'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac',
        'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
        'sweet_gas temperature, C',
        'sweet_gas CO2 mol frac', 'sweet_gas Methane mol frac', 'sweet_gas Ethane mol frac', 'sweet_gas Propane mol frac',
        'sweet_gas i-Butane mol frac', 'sweet_gas n-Butane mol frac', 'sweet_gas i-Pentane mol frac',
        'sweet_gas n-Pentane mol frac', 'sweet_gas H2S mol frac', 'sweet_gas H2O mol frac', 'sweet_gas MDEAmine mol frac',
        'rich_amine temperature, C', 'rich_amine CO2 mol frac', 'rich_amine Methane mol frac',
        'rich_amine Ethane mol frac', 'rich_amine Propane mol frac', 'rich_amine i-Butane mol frac',
        'rich_amine n-Butane mol frac', 'rich_amine i-Pentane mol frac', 'rich_amine n-Pentane mol frac',
        'rich_amine H2S mol frac', 'rich_amine H2O mol frac', 'rich_amine MDEAmine mol frac'
    ]
]

# Создаем X и y
X = prod_streams_physic[
    [
        'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight',
        'feed_gas Mass density, kg/m3', 'feed_gas actual liquid flow, m3/h', 'feed_gas actual vapour flow, m3/h',
        'feed_gas CO2 mol frac', 'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac',
        'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
        'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
        'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
        'lean_amine molecular weight', 'lean_amine Mass density, kg/m3', 'lean_amine actual liquid flow, m3/h',
        'lean_amine actual vapour flow, m3/h', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac',
        'lean_amine Ethane mol frac', 'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac',
        'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac',
        'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
        'sweet_gas CO2 mol frac', 'sweet_gas Methane mol frac', 'sweet_gas Ethane mol frac', 'sweet_gas Propane mol frac',
        'sweet_gas i-Butane mol frac', 'sweet_gas n-Butane mol frac', 'sweet_gas i-Pentane mol frac',
        'sweet_gas n-Pentane mol frac', 'sweet_gas H2S mol frac', 'sweet_gas H2O mol frac', 'sweet_gas MDEAmine mol frac',
        'rich_amine CO2 mol frac', 'rich_amine Methane mol frac', 'rich_amine Ethane mol frac',
        'rich_amine Propane mol frac', 'rich_amine i-Butane mol frac', 'rich_amine n-Butane mol frac',
        'rich_amine i-Pentane mol frac', 'rich_amine n-Pentane mol frac',
        'rich_amine H2S mol frac', 'rich_amine H2O mol frac', 'rich_amine MDEAmine mol frac'
    ]
]

y = prod_streams_physic.drop([
            'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight',
        'feed_gas Mass density, kg/m3', 'feed_gas actual liquid flow, m3/h', 'feed_gas actual vapour flow, m3/h',
        'feed_gas CO2 mol frac', 'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac',
        'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
        'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
        'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
        'lean_amine molecular weight', 'lean_amine Mass density, kg/m3', 'lean_amine actual liquid flow, m3/h',
        'lean_amine actual vapour flow, m3/h', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac',
        'lean_amine Ethane mol frac', 'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac',
        'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac',
        'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
        'sweet_gas CO2 mol frac', 'sweet_gas Methane mol frac', 'sweet_gas Ethane mol frac', 'sweet_gas Propane mol frac',
        'sweet_gas i-Butane mol frac', 'sweet_gas n-Butane mol frac', 'sweet_gas i-Pentane mol frac',
        'sweet_gas n-Pentane mol frac', 'sweet_gas H2S mol frac', 'sweet_gas H2O mol frac', 'sweet_gas MDEAmine mol frac',
        'rich_amine CO2 mol frac', 'rich_amine Methane mol frac', 'rich_amine Ethane mol frac',
        'rich_amine Propane mol frac', 'rich_amine i-Butane mol frac', 'rich_amine n-Butane mol frac',
        'rich_amine i-Pentane mol frac', 'rich_amine n-Pentane mol frac',
        'rich_amine H2S mol frac', 'rich_amine H2O mol frac', 'rich_amine MDEAmine mol frac'
], axis=1)

# Собираем трансформер столбцов
ct = make_column_transformer(
    (MinMaxScaler(), [
                'feed_gas temperature, C', 'feed_gas mass flow, kg/h', 'feed_gas molecular weight',
        'feed_gas Mass density, kg/m3', 'feed_gas actual liquid flow, m3/h', 'feed_gas actual vapour flow, m3/h',
        'feed_gas CO2 mol frac', 'feed_gas Methane mol frac', 'feed_gas Ethane mol frac', 'feed_gas Propane mol frac',
        'feed_gas i-Butane mol frac', 'feed_gas n-Butane mol frac', 'feed_gas i-Pentane mol frac',
        'feed_gas n-Pentane mol frac', 'feed_gas H2S mol frac', 'feed_gas H2O mol frac', 'feed_gas MDEAmine mol frac',
        'lean_amine temperature, C', 'lean_amine mass flow, kg/h',
        'lean_amine molecular weight', 'lean_amine Mass density, kg/m3', 'lean_amine actual liquid flow, m3/h',
        'lean_amine actual vapour flow, m3/h', 'lean_amine CO2 mol frac', 'lean_amine Methane mol frac',
        'lean_amine Ethane mol frac', 'lean_amine Propane mol frac', 'lean_amine i-Butane mol frac',
        'lean_amine n-Butane mol frac', 'lean_amine i-Pentane mol frac', 'lean_amine n-Pentane mol frac',
        'lean_amine H2S mol frac', 'lean_amine H2O mol frac', 'lean_amine MDEAmine mol frac',
        'sweet_gas CO2 mol frac', 'sweet_gas Methane mol frac', 'sweet_gas Ethane mol frac', 'sweet_gas Propane mol frac',
        'sweet_gas i-Butane mol frac', 'sweet_gas n-Butane mol frac', 'sweet_gas i-Pentane mol frac',
        'sweet_gas n-Pentane mol frac', 'sweet_gas H2S mol frac', 'sweet_gas H2O mol frac', 'sweet_gas MDEAmine mol frac',
        'rich_amine CO2 mol frac', 'rich_amine Methane mol frac', 'rich_amine Ethane mol frac',
        'rich_amine Propane mol frac', 'rich_amine i-Butane mol frac', 'rich_amine n-Butane mol frac',
        'rich_amine i-Pentane mol frac', 'rich_amine n-Pentane mol frac',
        'rich_amine H2S mol frac', 'rich_amine H2O mol frac', 'rich_amine MDEAmine mol frac'
    ])
)

# Создаем обучающую и тестовые выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Загружаем тренировочную выборку
ct.fit(X_train)

# Нормализуем данные
X_train_normal = ct.transform(X_train)
X_test_normal = ct.transform(X_test)
print(X_test_normal.shape)
print(y_test.shape)
y

(92304, 56)
(92304, 2)


,"sweet_gas temperature, C","rich_amine temperature, C"
226002,62.148492,55.875811
432331,24.128935,37.168606
92311,39.241484,48.766244
427449,58.157206,39.907921
393260,44.971243,43.876180
...,...,...
428317,36.189592,27.011813
207308,33.269666,41.154140
68949,21.366819,26.789920
24845,59.984320,65.991606


#### Нейронная сеть для определения температур продуктовых потоков

In [16]:
# Set random set
tf.random.set_seed(42)

# 1. Create a model
prod_temperature_model = tf.keras.Sequential([
    tf.keras.layers.Dense(56, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(2)
])

# 2. Compile the model
prod_temperature_model.compile(
    loss='mae',
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
)

# 3. Fit the model
prod_temperature_model.fit(X_train_normal, y_train, epochs=20)

Epoch 1/20
11538/11538 [==============================] - 25s 2ms/step - loss: 2.0303 - mae: 2.0303
Epoch 2/20
11538/11538 [==============================] - 29s 3ms/step - loss: 1.1527 - mae: 1.1527
Epoch 3/20
11538/11538 [==============================] - 30s 3ms/step - loss: 1.0099 - mae: 1.0099
Epoch 4/20
11538/11538 [==============================] - 29s 3ms/step - loss: 0.9328 - mae: 0.9328
Epoch 5/20
11538/11538 [==============================] - 29s 3ms/step - loss: 0.8709 - mae: 0.8709
Epoch 6/20
11538/11538 [==============================] - 30s 3ms/step - loss: 0.8313 - mae: 0.8313
Epoch 7/20
11538/11538 [==============================] - 41s 4ms/step - loss: 0.7963 - mae: 0.7963
Epoch 8/20
11538/11538 [==============================] - 39s 3ms/step - loss: 0.7692 - mae: 0.7692
Epoch 9/20
11538/11538 [==============================] - 41s 4ms/step - loss: 0.7576 - mae: 0.7576
Epoch 10/20
11538/11538 [==============================] - 42s 4ms/step - loss: 0.7404 - mae: 0.7404

In [18]:
prod_temperature_model.evaluate(X_test_normal)

2885/2885 [==============================] - 3s 1ms/step - loss: 0.0000e+00 - mae: 0.0000e+00


[0.0, 0.0]

In [19]:
y_pred = prod_temperature_model.predict(X_test_normal)

2885/2885 [==============================] - 6s 2ms/step


In [24]:
y_pred[3]

array([49.554695, 54.97244 ], dtype=float32)

In [21]:
y_test

,"sweet_gas temperature, C","rich_amine temperature, C"
317291,48.942342,59.338264
436955,24.308155,32.874179
27680,37.472216,50.852573
490943,49.849775,55.680787
349855,52.381815,47.649755
...,...,...
50877,53.187325,45.836792
107647,-67.279522,28.460279
378228,30.695414,26.004199
238158,30.135979,40.366815


#### Сохраняем модель для предсказания температур продуктовых потоков

In [27]:
prod_temperature_model.save('./models/amine_treatment/amine_treatment_prod_temperature_model.h5')

In [28]:
prod_temperature_model.save('./models/amine_treatment/amine_treatment_prod_temperature_model_sv')

INFO:tensorflow:Assets written to: ./models/amine_treatment/amine_treatment_prod_temperature_model_sv\assets


INFO:tensorflow:Assets written to: ./models/amine_treatment/amine_treatment_prod_temperature_model_sv\assets


In [1]:
columns = ['aaaa', 'bbbb', 'cccc', 'oooo']

In [3]:
col = [*columns, 'jjjj']

In [4]:
col

['aaaa', 'bbbb', 'cccc', 'oooo', 'jjjj']